In [1]:
# Same as direct inference except use the "Translated JSON Files" folder

# Romance

In [3]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "romance_translated"
method = "direct_inference"
model = "gpt-4o"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "es": "'{claim}' ¿Es esta afirmación «Verdadera», «Casi verdadera», «Medio verdadera», «Casi falsa» o «Falsa»? Debe asegurarse de que su respuesta final comience con «Respuesta final:» seguida de «Verdadero», «Casi verdadero», «Medio verdadero», «Casi falso» o «Falso».",
    "it": "'{claim}' Questa affermazione è “vera”, “quasi vera”, “mezza vera”, “quasi falsa” o “falsa”? Assicurati che la tua risposta finale inizi con “Risposta finale:” seguita da “Vero”, “Quasi vero”, “Mezzo vero”, “Quasi falso” o “Falso”.",
    "pt": "'{claim}' Esta afirmação é “Verdadeira”, “Quase Verdadeira”, “Meia Verdadeira”, “Quase Falsa” ou “Falsa”? Certifique-se de que sua resposta final comece com “Resposta Final:” seguida de “Verdadeiro”, “Quase Verdadeiro”, “Meio Verdadeiro”, “Quase Falso” ou “Falso”.",
    "fr": "'{claim}' Cette affirmation est-elle « vraie », « en grande partie vraie », « à moitié vraie », « en grande partie fausse » ou « fausse » ? Vous devez vous assurer que votre réponse finale commence par « Réponse finale: » suivie de « Vrai », « En grande partie vrai », « À moitié vrai », « En grande partie faux » ou « Faux »."
}

final_answer_mappings = {
    "es": {
        "respuesta final: verdadero": "true", "respuesta final: verdadera": "true",
        "respuesta final: casi verdadero": "mostly true", "respuesta final: casi verdadera": "mostly true",
        "respuesta final: medio verdadero": "half true", "respuesta final: medio verdadera": "half true",
        "respuesta final: casi falso": "mostly false", "respuesta final: casi falsa": "mostly false",
        "respuesta final: falso": "false", "respuesta final: falsa": "false",
        "verdadero": "true", "verdadera": "true",
        "casi verdadero": "mostly true", "casi verdadera": "mostly true",
        "medio verdadero": "half true", "medio verdadera": "half true",
        "casi falso": "mostly false", "casi falsa": "mostly false",
        "falso": "false", "falsa": "false"
    },
    "it": {
        "risposta finale: vero": "true", "risposta finale: vera": "true",
        "risposta finale: quasi vero": "mostly true", "risposta finale: quasi vera": "mostly true",
        "risposta finale: mezzo vero": "half true", "risposta finale: mezza vera": "half true",
        "risposta finale: quasi falso": "mostly false", "risposta finale: quasi falsa": "mostly false",
        "risposta finale: falso": "false", "risposta finale: falsa": "false",
        "vero": "true", "vera": "true",
        "quasi vero": "mostly true", "quasi vera": "mostly true",
        "mezzo vero": "half true", "mezza vera": "half true",
        "quasi falso": "mostly false", "quasi falsa": "mostly false",
        "falso": "false", "falsa": "false"
    },
    "pt": {
        "resposta final: verdadeiro": "true", "resposta final: verdadeira": "true",
        "resposta final: quase verdadeiro": "mostly true", "resposta final: quase verdadeira": "mostly true",
        "resposta final: meio verdadeiro": "half true", "resposta final: meia verdadeira": "half true",
        "resposta final: quase falso": "mostly false", "resposta final: quase falsa": "mostly false",
        "resposta final: falso": "false", "resposta final: falsa": "false",
        "verdadeiro": "true", "verdadeira": "true",
        "quase verdadeiro": "mostly true", "quase verdadeira": "mostly true",
        "meio verdadeiro": "half true", "meia verdadeira": "half true",
        "quase falso": "mostly false", "quase falsa": "mostly false",
        "falso": "false", "falsa": "false"
    },
    "fr": {
        "réponse finale: vrai": "true", "réponse finale: vraie": "true",
        "réponse finale: plus ou moins vrai": "mostly true", "réponse finale: plus ou moins vraie": "mostly true",
        "réponse finale: moitié vrai": "half true", "réponse finale: moitié vraie": "half true",
        "réponse finale: plus ou moins faux": "mostly false", "réponse finale: plus ou moins fausse": "mostly false",
        "réponse finale: faux": "false", "réponse finale: fausse": "false",
        "vrai": "true", "vraie": "true",
        "plus ou moins vrai": "mostly true", "plus ou moins vraie": "mostly true",
        "moitié vrai": "half true", "moitié vraie": "half true",
        "plus ou moins faux": "mostly false", "plus ou moins fausse": "mostly false",
        "faux": "false", "fausse": "false"
    }
}

def get_gpt_response(claim, language, model=model):
    try:
        user_prompt = prompts[language].format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Z\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(respuesta final:|risposta finale:|resposta final:|réponse finale:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_gpt_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Pour évaluer cette affirmation, nous devons examiner les dépenses de santé par habitant (per capita) de chaque pays. Selon les données de la Banque mondiale et d'autres sources fiables, les dépenses de santé par habitant en 2021 étaient approximativement les suivantes :

- Nigeria : environ 70 USD par habitant.
- Afrique du Sud : environ 500 USD par habitant.

Si nous comparons ces chiffres, nous constatons que l'Afrique du Sud dépense environ 7 fois plus par habitant que le Nigeria en matière de santé (500 / 70 ≈ 7.14).

Réponse finale: En grande partie vrai.
Final Answer: True, Actual Answer: False
Wrong response
Model Output: Pour évaluer l'exactitude de l'affirmation selon laquelle "le blocage du quart a été supprimé de la constitution sénégalaise depuis 2006", il est important de comprendre ce que signifie "le blocage du quart" et de vérifier les modifications constitutionnelles pertinentes au Sénégal.

Le "blocage du quart" fait référence à une disposition constitut

# Indo-Aryan

In [4]:
import pandas as pd
import openai
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "indo_aryan_translated"
method = "direct_inference"
model = "gpt-4o"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts
prompts = {
    "hi": "'{claim}' क्या यह कथन «सत्य», «अधिकांशतः सत्य», «आधा सत्य», «अधिकांशतः असत्य» या «असत्य» है? आपको हमेशा यह सुनिश्चित करना चाहिए कि आपका अंतिम उत्तर «अंतिम उत्तर:» के साथ शुरू हो और उसके बाद «सत्य», «अधिकांशतः सत्य», «आधा सत्य», «अधिकांशतः असत्य» या «असत्य» हो।",
    "bn": "'{claim}' এই বক্তব্যটি «সত্য», «অধিকাংশ সত্য», «অর্ধসত্য», «অধিকাংশ মিথ্যা» বা «মিথ্যা» কি? সর্বদা নিশ্চিত করুন যে আপনার চূড়ান্ত উত্তর «চূড়ান্ত উত্তর:» দিয়ে শুরু হয় এবং তারপরে «সত্য», «অধিকাংশ সত্য», «অর্ধসত্য», «অধিকাংশ মিথ্যা» বা «মিথ্যা»।",
    "pa": "'{claim}' ਕੀ ਇਹ ਬਿਆਨ «ਸੱਚ», «ਜਿਆਦਾਤਰ ਸੱਚ», «ਅੱਧਾ ਸੱਚ», «ਜਿਆਦਾਤਰ ਝੂਠ» ਜਾਂ «ਝੂਠ» ਹੈ? ਤੁਹਾਨੂੰ ਹਮੇਸ਼ਾਂ ਇਹ ਯਕੀਨੀ ਬਣਾਉਣਾ ਚਾਹੀਦਾ ਹੈ ਕਿ ਤੁਹਾਡਾ ਅੰਤਿਮ ਜਵਾਬ «ਅੰਤਿਮ ਜਵਾਬ:» ਨਾਲ ਸ਼ੁਰੂ ਹੋਵੇ ਅਤੇ ਫਿਰ «ਸੱਚ», «ਜਿਆਦਾਤਰ ਸੱਚ», «ਅੱਧਾ ਸੱਚ», «ਜਿਆਦਾਤਰ ਝੂਠ» ਜਾਂ «ਝੂਠ» ਹੋਵੇ।",
    "gu": "'{claim}' શું આ નિવેદન «સત્ય», «મોટાભાગનું સત્ય», «અડધું સત્ય», «મોટાભાગનું ખોટું» કે «ખોટું» છે? તમે હંમેશાં ખાતરી કરવી જોઈએ કે તમારો અંતિમ જવાબ «અંતિમ જવાબ:» થી શરૂ થાય અને પછી «સત્ય», «મોટાભાગનું સત્ય», «અડધું સત્ય», «મોટાભાગનું ખોટું» કે «ખોટું» આવે।",
    "mr": "'{claim}' हे विधान «सत्य», «जास्तीत जास्त सत्य», «अर्धसत्य», «जास्तीत जास्त असत्य» किंवा «असत्य» आहे का? तुम्ही नेहमी हे सुनिश्चित केले पाहिजे की तुमचे अंतिम उत्तर «अंतिम उत्तर:» ने सुरू होते आणि त्यानंतर «सत्य», «जास्तीत जास्त सत्य», «अर्धसत्य», «जास्तीत जास्त असत्य» किंवा «असत्य» येते."
}

def get_gpt_response(claim, language, model=model):
    try:
        user_prompt = prompts[language].format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_gpt_response(claim, language)
    
    print(f"Model Output: {output}")
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'correct': False,  # Placeholder
        'inconclusive': True  # Placeholder
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    summary['languages'][language]['inconclusive'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: চূড়ান্ত উত্তর: মিথ্যা

George Floyd একজন ব্যক্তি ছিলেন যিনি ২০২০ সালে মিনিয়াপোলিস, মার্কিন যুক্তরাষ্ট্রে পুলিশ হেফাজতে মারা যান। তিনি একজন ফটোগ্রাফার ছিলেন না এবং তার মৃত্যুর সাথে সম্পর্কিত কোনো ছবি তার দ্বারা তোলা হয়নি। সুতরাং, উক্ত বক্তব্যটি মিথ্যা।
Model Output: अंतिम उत्तर: असत्य
Model Output: अंतिम उत्तर: असत्य
Model Output: ਅੰਤਿਮ ਜਵਾਬ: ਝੂਠ
Model Output: চূড়ান্ত উত্তর: মিথ্যা
Model Output: इस कथन का मूल्यांकन करने के लिए हमें यह जानना होगा कि क्या वर्तमान में इजरायली प्रधानमंत्री का बेटा सेना में सेवा कर रहा है या नहीं। इजरायल में अनिवार्य सैन्य सेवा होती है, और अधिकांश नागरिकों को सेना में सेवा करनी होती है। 

हालांकि, यह जानकारी समय-समय पर बदल सकती है और वर्तमान स्थिति की पुष्टि के लिए नवीनतम स्रोतों की आवश्यकता होती है। 

यदि हम मान लें कि इजरायली प्रधानमंत्री का बेटा वर्तमान में सेना में सेवा कर रहा है, तो यह कथन सत्य होगा। अन्यथा, यह असत्य होगा।

अंतिम उत्तर: आधा सत्य
Model Output: «চূড়ান্ত উত্তর: মিথ্যা»
Model Output: अंतिम उत्तर: अधिकांशतः असत्य
Model Out

# Kartvelian

In [6]:
import pandas as pd
import openai
import json
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "kartvelian_translated"
method = "direct_inference"
model = "gpt-4o"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts
prompts = {
    "ka": "'{claim}' არის ეს პრეტენზია «მართალია», «ძირითადად მართალია», «ნახევრად მართალია», «ძირითადად მცდარი» ან «მცდარი»? თქვენ ყოველთვის უნდა დარწმუნდეთ, რომ თქვენს საბოლოო პასუხს აქვს პრეფიქსი „საბოლოო პასუხი:“ რასაც მოჰყვება „მართალი“, „ძირითადად მართალია“, „ნახევრად მართალია“, „ძირითადად მცდარი“ ან „მცდარი“."
}

def get_gpt_response(claim, language, model=model):
    try:
        user_prompt = prompts[language].format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_gpt_response(claim, language)
    
    print(f"Model Output: {output}")
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'correct': False,  # Placeholder
        'inconclusive': True  # Placeholder
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    summary['languages'][language]['inconclusive'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: იმისათვის, რომ ზუსტად შევაფასოთ პრეტენზია "Partnership Fund is a loss for 4 years, GEL 700 million", საჭიროა დეტალური ფინანსური მონაცემების გადამოწმება. ეს მოიცავს ფონდის ფინანსური ანგარიშების ანალიზს ბოლო ოთხი წლის განმავლობაში, რათა დადგინდეს, თუ ნამდვილად არის თუ არა ფონდი ზარალში და თუ ეს ზარალი შეადგენს 700 მილიონ ლარს.

თუ ფონდის ფინანსური ანგარიშები აჩვენებს, რომ ბოლო ოთხი წლის განმავლობაში ფონდი იყო ზარალში და ეს ზარალი შეადგენს 700 მილიონ ლარს, მაშინ პრეტენზია იქნება „მართალი“. თუ ზარალი არის, მაგრამ არა 700 მილიონი ლარი, მაშინ პრეტენზია შეიძლება იყოს „ძირითადად მართალია“ ან „ნახევრად მართალია“. თუ ფონდი არ არის ზარალში ან ზარალი არ შეადგენს 700 მილიონ ლარს, მაშინ პრეტენზია იქნება „ძირითადად მცდარი“ ან „მცდარი“.

საბოლოო პასუხი: ამ ეტაპზე, დამატებითი ფინანსური მონაცემების გარეშე, ვერ ვიტყვი ზუსტად. საჭიროა დამატებითი ინფორმაცია.
Model Output: იმისათვის, რომ განვსაზღვროთ, არის თუ არა ეს პრეტენზია მართალი, საჭიროა 2013 წლის პირველი კვარტლის ეკონომიკური მონაცემების 

# Slavic

In [7]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "slavic_translated"
method = "direct_inference"
model = "gpt-4o"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "ru": "'{claim}' является ли это утверждение «правдивым», «в основном правдивым», «наполовину правдивым», «в основном ложным» или «ложным»? Вы должны всегда убедиться, что ваш окончательный ответ начинается с «Окончательный ответ:» и далее следует «Правда», «В основном правда», «Наполовину правда», «В основном ложь» или «Ложь».",
    "pl": "'{claim}' czy to stwierdzenie jest „prawdziwe”, „przeważnie prawdziwe”, „w połowie prawdziwe”, „przeważnie fałszywe” czy „fałszywe”? Musisz zawsze upewnić się, że twoja ostateczna odpowiedź zaczyna się od „Ostateczna odpowiedź:” a następnie „Prawda”, „Przeważnie prawda”, „W połowie prawda”, „Przeważnie fałsz” lub „Fałsz”.",
    "sr": "'{claim}' да ли је ова изјава „истинита”, „углавном истинита”, „полуистинита”, „углавном лажна” или „лажна”? Морате увек осигурати да ваш коначан одговор почиње са „Коначан одговор:” и затим следи „Истина”, „Углавном истина”, „Полуистина”, „Углавном лаж” или „Лаж”."
}

final_answer_mappings = {
    "ru": {
        "окончательный ответ: правда": "true",
        "окончательный ответ: в основном правда": "mostly true",
        "окончательный ответ: наполовину правда": "half true",
        "окончательный ответ: в основном ложь": "mostly false",
        "окончательный ответ: ложь": "false",
        "правда": "true",
        "в основном правда": "mostly true",
        "наполовину правда": "half true",
        "в основном ложь": "mostly false",
        "ложь": "false"
    },
    "pl": {
        "ostateczna odpowiedź: prawda": "true",
        "ostateczna odpowiedź: przeważnie prawda": "mostly true",
        "ostateczna odpowiedź: w połowie prawda": "half true",
        "ostateczna odpowiedź: przeważnie fałsz": "mostly false",
        "ostateczna odpowiedź: fałsz": "false",
        "prawda": "true",
        "przeważnie prawda": "mostly true",
        "w połowie prawda": "half true",
        "przeważnie fałsz": "mostly false",
        "fałsz": "false"
    },
    "sr": {
        "коначан одговор: истина": "true",
        "коначан одговор: углавном истина": "mostly true",
        "коначан одговор: полуистина": "half true",
        "коначан одговор: углавном лаж": "mostly false",
        "коначан одговор: лаж": "false",
        "истина": "true",
        "углавном истина": "mostly true",
        "полуистина": "half true",
        "углавном лаж": "mostly false",
        "лаж": "false"
    }
}

def get_gpt_response(claim, language, model=model):
    try:
        user_prompt = prompts[language].format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-Zа-яА-Я\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(окончательный ответ:|остateczna odpowiedź:|коначан одговор:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_gpt_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Коначан одговор: Полуистина

Образложење: Изјава сугерише да постоје различити датуми за акцизе у различитим европским земљама у зависности од квалитета нафтних деривата. Иако је тачно да различите земље могу имати различите акцизне стопе и регулативе за нафтне деривате, изјава није прецизна у погледу "различитих датума" и "квалитета" производа. Акцизе се обично односе на врсту и количину производа, а не на датуме.
Final Answer: True, Actual Answer: Mostly false
Wrong response
Model Output: Коначан одговор: Истина
Final Answer: True, Actual Answer: False
Wrong response
Model Output: Для того чтобы определить, является ли утверждение правдивым, необходимо проверить, соответствуют ли приведенные данные реальным статистическим данным о зарплатах в январе. В данном случае утверждается, что медианная зарплата составляет 112 тысяч тенге, а модальная зарплата — 58 тысяч тенге.

Если предположить, что эти данные взяты из достоверного источника, то утверждение может быть правдивым

# Turkic

In [8]:
import pandas as pd
import openai
import json
import os
import re
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the OpenAI API key from the environment variable
openai.api_key = os.getenv('OPENAI_API_KEY')

# Define the prompting method and language family
language_family = "turkic_translated"
method = "direct_inference"
model = "gpt-4o"

# Load data
input_file_path = f'../../Translated JSON Files/{language_family}.json'
output_file_path = f"{language_family}_{model}_{method}.csv"
summary_file_path = f"{language_family}_{model}_{method}_summary.json"

if os.path.exists(input_file_path):
    with open(input_file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
else:
    raise FileNotFoundError(f"File not found: {input_file_path}")

claims = [item['claim'] for item in data]
labels = [item['label'] for item in data]
languages = [item['language'] for item in data]
sites = [item['site'] for item in data]

df = pd.DataFrame({
    'claim': claims,
    'label': labels,
    'language': languages,
    'site': sites
})

# Language-specific user prompts and final answer mappings
prompts = {
    "tr": "'{claim}' bu ifade «doğru», «çoğunlukla doğru», «yarı doğru», «çoğunlukla yanlış» veya «yanlış» mı? «Nihai Cevap: Doğru», «Nihai Cevap: Çoğunlukla Doğru», «Nihai Cevap: Yarı Doğru», «Nihai Cevap: Çoğunlukla Yanlış» veya «Nihai Cevap: Yanlış» şeklinde cevap verin.",
    "az": "'{claim}' bu ifadə «doğru», «əsasən doğru», «yarı doğru», «əsasən yanlış» və ya «yanlış»dır? «Son Cavab: Doğru», «Son Cavab: Əsasən Doğru», «Son Cavab: Yarı Doğru», «Son Cavab: Əsasən Yanlış» və ya «Son Cavab: Yanlış» olaraq cavab verin."
}

final_answer_mappings = {
    "tr": {
        "nihai cevap: doğru": "true",
        "nihai cevap: çoğunlukla doğru": "mostly true",
        "nihai cevap: yarı doğru": "half true",
        "nihai cevap: çoğunlukla yanlış": "mostly false",
        "nihai cevap: yanlış": "false",
        "doğru": "true",
        "çoğunlukla doğru": "mostly true",
        "yarı doğru": "half true",
        "çoğunlukla yanlış": "mostly false",
        "yanlış": "false"
    },
    "az": {
        "son cavab: doğru": "true",
        "son cavab: əsasən doğru": "mostly true",
        "son cavab: yarı doğru": "half true",
        "son cavab: əsasən yanlış": "mostly false",
        "son cavab: yanlış": "false",
        "doğru": "true",
        "əsasən doğru": "mostly true",
        "yarı doğru": "half true",
        "əsasən yanlış": "mostly false",
        "yanlış": "false"
    }
}

def get_gpt_response(claim, language, model=model):
    try:
        user_prompt = prompts[language].format(claim=claim)
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "user", "content": user_prompt}
            ],
            temperature=0,
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error getting GPT response: {e}")
        return ""

def clean_output(output):
    cleaned_output = re.sub(r'[^a-zA-ZəƏıİşŞöÖüÜğĞçÇ\s:]', '', output)
    return cleaned_output.lower()

def extract_final_answer(cleaned_output, language):
    try:
        final_answer = re.search(r'(nihai cevap:|son cavab:)\s*([\w\s]+)', cleaned_output)
        if final_answer:
            response = final_answer.group(2).strip()
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in response:
                    return answer
        else:
            for keyword, answer in final_answer_mappings[language].items():
                if keyword in cleaned_output:
                    return answer
    except Exception as e:
        print(f"Error extracting final answer: {e}")
    return None

# Initialize or load existing outputs and summary
if os.path.exists(output_file_path):
    output_df = pd.read_csv(output_file_path)
    outputs = output_df.to_dict('records')
else:
    outputs = []

if os.path.exists(summary_file_path):
    with open(summary_file_path, 'r', encoding='utf-8') as file:
        summary = json.load(file)
else:
    summary = {
        'correct': 0,
        'wrong': 0,
        'inconclusive': 0,
        'total': 0,
        'languages': {}
    }

# Process claims and update files iteratively
for index, row in df.iterrows():
    if any(output['claim'] == row['claim'] for output in outputs):
        continue  # Skip already processed claims

    claim = row['claim']
    label = row['label']
    language = row['language']
    
    if language not in prompts:  # Skip unsupported languages
        continue

    output = get_gpt_response(claim, language)
    
    print(f"Model Output: {output}")
    
    cleaned_output = clean_output(output)
    final_answer = extract_final_answer(cleaned_output, language)
    
    if final_answer is None:
        print("Inconclusive response")
        summary['inconclusive'] += 1
    else:
        print(f"Final Answer: {final_answer.capitalize()}, Actual Answer: {label.capitalize()}")
        if final_answer == label.lower():
            print("Correct response")
            summary['correct'] += 1
        else:
            print("Wrong response")
            summary['wrong'] += 1
    
    # Save outputs
    output_record = {
        'claim': claim,
        'label': label,
        'language': language,
        'output': output,
        'final_answer': final_answer,
        'correct': final_answer == label.lower() if final_answer else False,
        'inconclusive': final_answer is None
    }
    outputs.append(output_record)
    
    # Update language summary
    if language not in summary['languages']:
        summary['languages'][language] = {'correct': 0, 'wrong': 0, 'inconclusive': 0, 'total': 0}
    summary['languages'][language]['total'] += 1
    summary['total'] += 1
    if final_answer is None:
        summary['languages'][language]['inconclusive'] += 1
    elif final_answer == label.lower():
        summary['languages'][language]['correct'] += 1
    else:
        summary['languages'][language]['wrong'] += 1

    # Save results to CSV iteratively
    pd.DataFrame(outputs).to_csv(output_file_path, index=False, encoding='utf-8')

    # Save summary to JSON iteratively
    with open(summary_file_path, 'w', encoding='utf-8') as file:
        json.dump(summary, file, ensure_ascii=False, indent=4)

print(f"Results saved to {output_file_path} and {summary_file_path}")
print(f"Summary: {summary}")


Model Output: Nihai Cevap: Yanlış
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Son Cavab: Əsasən Yanlış
Final Answer: Mostly false, Actual Answer: Mostly false
Correct response
Model Output: Nihai Cevap: Yanlış
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Nihai Cevap: Doğru
Final Answer: True, Actual Answer: Half true
Wrong response
Model Output: Nihai Cevap: Yanlış
Final Answer: False, Actual Answer: Mostly false
Wrong response
Model Output: Nihai Cevap: Yanlış
Final Answer: False, Actual Answer: Mostly true
Wrong response
Model Output: Nihai Cevap: Doğru
Final Answer: True, Actual Answer: Half true
Wrong response
Model Output: Nihai Cevap: Yarı Doğru
Final Answer: True, Actual Answer: Mostly false
Wrong response
Model Output: Bu ifadenin doğruluğunu değerlendirmek için 2002 yılı ve mevcut yıl arasındaki Milli Eğitim Bakanlığı (MoNE) bütçesi verilerini karşılaştırmak gerekmektedir. Ancak, elimde bu veriler olmadığı için